# Linear Model with AIC feature engineering

The linear method makes use of the log backward return (log price difference) to predict foward return, thus decide log price.

Training:
1) Ridge regression: on 30 features
2) PC regression: pca on 30 features then perform ols

Feature: 10 stocks, each with 3 backward return (say, 3min, 7min, 10min, see rolling cross cor)

Response: do 10 regression, each for 1 stock. 

## Data Preparation

In [1]:
import os
import datetime
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

log_pr = pd.read_pickle("../data/log_price.df")
volu = pd.read_pickle("../data/volume_usd.df")

In [10]:
# Data Preparation
def logr(dta,min):
    '''
    Input
    dta: pandas dataframe nxp
    min: backward length
    Return
    log return: pandas dataframe
    '''
    return dta.diff(min,0).dropna()

In [58]:
x_begin_idx = log_pr.index[30]
x_end_idx = log_pr.index[-31]
y_begin_idx = log_pr.index[60]

x = pd.concat((logr(log_pr,2)[x_begin_idx:x_end_idx],
    logr(log_pr,3)[x_begin_idx:x_end_idx],
    logr(log_pr,7)[x_begin_idx:x_end_idx]),axis=1)
y = log_pr.diff(30)[y_begin_idx:]
y = y.set_index(x.index)

from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.3)
xtrain,xval,ytrain,yval = train_test_split(xtrain,ytrain,test_size=0.3)

In [67]:
# preprocessing
def remove_outliers(dta):
    # Compute the mean and interquartile range
    mean = dta.mean(0)
    iqr = dta.quantile([0.25, 0.75], axis=0).diff().T.iloc[:, 1]
    # Replace entries that are more than 10 times the IQR
    # away from the mean with NaN (denotes a missing entry)
    mask = np.abs(dta) > mean + 10 * iqr
    treated = dta.copy()
    treated[mask] = np.nan
    return treated

## Ridge Regression

In [ ]:
#ridge_trainx
#ridge_trainy

from sklearn.linear_model import Ridge

rr = Ridge().fit(xtrain,ytrain)



## PC Regression

In [ ]:
# pca_trainx
# pca_trainy

from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.linear_model import LinearRegression

pca = PCA()
pca_trainx = pca.fit_transform(scale(trainx))

lr = LinearRegression().fit(pca_trainx,pca_trainy)


## Evaluation